In [1]:
import os
from copy import copy

import sympy

from sympy.printing.ccode import C99CodePrinter
from sympy.codegen.ast import real, float32

import sympy_utils
from sympy_utils import short_latex, ShortLatexPrinter, matsym, vec, dynvec, SympyDumpable

import model
from model import Model
import algorithms
from algorithms import StaticLinearization, DynamicLinearization

sympy.init_session(latex_printer=short_latex)

IPython console for SymPy 1.4 (Python 3.7.3-64-bit) (ground types: gmpy)

These commands were executed:
>>> from __future__ import division
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.4/



In [2]:
# import model data from file
mapping = [
    ('full_1', Model),
    ('full_1_static', StaticLinearization),
    ('full_1_dynamic', DynamicLinearization),
    ('full_2', Model),
    ('full_2_static', StaticLinearization),
    ('full_2_dynamic', DynamicLinearization),
    ('full_2_JPTD', Model),
    ('full_2_JPTD_static', StaticLinearization),
    ('full_2_JPTD_dynamic', DynamicLinearization),
    ('simplified', Model),
    ('simplified_static', StaticLinearization),
    ('simplified_dynamic_1', DynamicLinearization),
    ('simplified_dynamic_2', DynamicLinearization),
]

models = SympyDumpable.load('generated_models.json', mapping)

In [3]:
class ArraySymbolsCodePrinter(C99CodePrinter):
    """
    Code printer that allows to convert symbols that belong to given matrix
    to indexing operations on that matrix, instead of using their explicit names.
    For example:
        we have a vector q = [x, y, theta]
        in all equations the symbols x, y, theta are used
        but we want to have an array argument to our function
        we do:
            printer = ArraySymbolsCodePrinter({'q': Matrix([x, y, theta])})
        now each time when there is theta in equations it will be printed as 'q[2]'
    """
    
    def __init__(self, symbols_mappings, *args, **kwargs):
        """
        symbols_mappings - dictionary: {'vector_name': vector_symbols}
        
        "hidden" kwargs:
            unevaluated_indexing=True
        """
        self.symbols_mappings = symbols_mappings
        self.unevaluated_indexing = kwargs.pop('unevaluated_indexing', True)
        # prevent 'eating up' settings
        settings = copy(kwargs.get('settings', {}))
        kwargs['settings'] = settings
        super().__init__(*args, **kwargs)

    def try_get_replaced(self, expr):
        # check each available mapping
        for name, matrix in self.symbols_mappings.items():
            # check all the elements of this matrix to see if it equals expr
            for row in range(matrix.shape[0]):
                for col in range(matrix.shape[1]):
                    if expr == matrix[row, col]:
                        ## IMPORTANT: we assume row-major order
                        if self.unevaluated_indexing:
                            return '{}[{}*{}+{}]'.format(name, row, matrix.shape[1], col)
                        else:
                            return '{}[{}]'.format(name, row*matrix.shape[1] + col)
        return None

    def _print_Function(self, expr):
        replaced = self.try_get_replaced(expr)
        if replaced is not None:
            return replaced
        return super()._print_Function(expr)                  

    def _print_Symbol(self, expr):
        replaced = self.try_get_replaced(expr)
        if replaced is not None:
            return replaced
        return super()._print_Symbol(expr)                  

    
def expand_matrix_symbol(mat):
    "Convert MatrixSymbol to a Matrix of Symbols with auto-generated names."
    syms = []
    for row in range(mat.shape[0]):
        syms.append([])
        for col in range(mat.shape[1]):
            syms[row].append(Symbol('%s_%d_%d' % (mat.name, row, col)))
    return Matrix(syms)

def generate_function(name, printer, out_expr, assign_to, input_args, 
                      indent=4, unevaluated_args_dims=True, use_cse=False):
    """
    Generate C code for function to compute `out_expr`.
    
    Args:
        name       - function name
        printer    - an instance of ArraySymbolsCodePrinter with proper mappings
        out_expr   - expression that defines the result 
        assign_to  - MatrixSymbol that is equivalent to `out_expr`
        input_args - list of function input arguments (1st arg is the output),
                     these should be MatrixSymbols that are later converted to
                     Matrix with expand_matrix_symbol();
                     user must supply all the required arguments
                     
        indent     - number of spaces used per indentation level
        unevaluated_args_dims - print array types dimensions as unvaluated 
                     multiplications of constants, which better visualizes their
                     corresponding matrix dimensions
        use_cse    - pefrom common subexpression elimination to avoid unnecessary
                     repeated computations in the code
    """
    
    def arg_decl(arg, const=False):
        if unevaluated_args_dims:
            return '%sfloat %s[%d*%d]' % ('const ' if const else '', printer.doprint(arg), *arg.shape)
        else:
            return '%sfloat %s[%d]' % ('const ' if const else '', printer.doprint(arg), arg.shape[0]*arg.shape[1])
            
    
    out = arg_decl(assign_to)
    inputs = [arg_decl(i, const=True) for i in input_args]

    subs = {}
    for iarg in input_args:
        subs[iarg] = expand_matrix_symbol(iarg)
    
    # replace the expression with expanded one
    out_expr = out_expr.subs(subs).doit()
    # update substitution for the printer
    for arg, sub in subs.items():
        if arg.name not in printer.symbols_mappings.keys():
            printer.symbols_mappings[arg.name] = sub
    
    func_decl = 'void %s(%s)' % (name, ', '.join([out] + inputs))
    code_lines = []
    
    if use_cse:
        # common subexpression elimination
        def decl(lhs, rhs):
            return 'float %s = %s;' % (printer.doprint(lhs), printer.doprint(rhs))

        cse_subs, cse_reduced = cse([out_expr], optimizations='basic')
        cse_reduced, = cse_reduced  # cse returns a list

        # generate intermediate variables declarations with assignment
        for lhs, rhs in cse_subs:
            code_lines.append(decl(lhs, rhs))
            
        out_expr = cse_reduced

    # generate code for the simplified version of expr
    code_lines.append('')
    code_lines += printer.doprint(out_expr, assign_to=assign_to).split('\n')
    
    return '%s {\n%s\n}' % (func_decl, '\n'.join((indent * ' ' + line for line in code_lines)))

def function_declaration(function_code):
    return function_code[:function_code.find('{')].strip() + ';'

def generate_file_pair(base_dir, base_filename, functions, includes=[], private_includes=[]):
    path = os.path.join(base_dir, base_filename)
    with open(path + '.c', 'w') as f:
        f.write('#include "%s.h"\n\n'  % base_filename)
        f.write('\n')    
        for inc in private_includes:
            f.write(inc + '\n')
        f.write('\n')    
        for func in functions:
            f.write(func + '\n')
    with open(path + '.h', 'w') as f:
        f.write('#pragma once\n\n')
        for inc in includes:
            f.write(inc + '\n')
        f.write('\n')    
        for func in functions:
            f.write(function_declaration(func) + '\n')

In [4]:
# def model_kinematics_code(model, printer_kwargs, unevaluated_args_dims=True):
    

def static_lin_code(model, algorithm, printer_kwargs, generator_kwargs, prefix=''):
    m, alg = model, algorithm
    assert hasattr(alg, 'D_inv') and alg.D_inv is not None, 'D matrix in the algorithm has no inverse!'

    funcs_code = {}
    
    # first the control algorithm function (regulator)
    K = MatrixSymbol('K', len(alg.h), len(alg.h))
    d_hd, hd = matsym('d_hd', alg.h), matsym('hd', alg.h)
    h_sym = matsym('h', alg.h)
    q_sym = matsym('q', m.q)

    u = d_hd - K * (alg.h - hd)
    u_sym = matsym('u', u)

    mappings = {'h': alg.h, 'q': m.q}
    printer = ArraySymbolsCodePrinter(mappings, **printer_kwargs)

    name = 'feedback_u'
    code = generate_function(name=prefix + name, printer=printer, 
                             out_expr=u, assign_to=u_sym, 
                             input_args=[K, q_sym, h_sym, hd, d_hd],
                             **generator_kwargs) 
    funcs_code[name] = code

    # now generate the static linearization function
    mappings = {'q': m.q}
    printer = ArraySymbolsCodePrinter(mappings, **printer_kwargs)

    eta = alg.D_inv * u_sym
    eta_sym = matsym('eta', m.eta)

    name = 'static_lin_eta'
    code = generate_function(name=prefix + name, 
                             printer=printer, 
                             out_expr=eta, assign_to=eta_sym, 
                             input_args=[u_sym, q_sym],
                             **generator_kwargs) 
    funcs_code[name] = code


    # generate a function for computing the determinant of D matrix
    det_D = alg.D.det().simplify()
    det_sym = Symbol('det_D')

    mappings = {'q': m.q}
    # mappings = {}
    printer = ArraySymbolsCodePrinter(mappings, **printer_kwargs)

    name = 'static_lin_det_D'
    code = """
float %s(const float q[]) {
    float %s
    return det_D;
}
    """.strip() % (prefix + name, printer.doprint(det_D, assign_to=det_sym))
    funcs_code[name] = code
    
    return funcs_code

def dynamic_lin_code(model, algorithm, printer_kwargs, generator_kwargs, prefix=''):
    m, alg = model, algorithm
    assert hasattr(alg, 'Kdd_inv') and alg.Kdd_inv is not None, 'Kdd matrix in the algorithm has no inverse!'
    
    funcs_code = {}

    # first the control algorithm function (regulator)
    K1 = MatrixSymbol('K1', len(alg.h), len(alg.h))
    K2 = MatrixSymbol('K2', len(alg.h), len(alg.h))
    d2_hd, d_hd, hd = matsym('d2_hd', alg.h), matsym('d_hd', alg.h), matsym('hd', alg.h)
    h_sym, d_h = matsym('h', alg.h), matsym('d_h', alg.h)
    q_sym = matsym('q', m.q)

    v = d2_hd - K1 * (d_h - d_hd) - K2 * (alg.h - hd)
    v_sym = matsym('v', v)
    
    mappings = {'h': alg.h, 'q': m.q}
    printer = ArraySymbolsCodePrinter(mappings, **printer_kwargs)

    name = 'feedback_v'
    code = generate_function(name=prefix + name, printer=printer, 
                             out_expr=v, assign_to=v_sym, 
                             input_args=[K1, K2, q_sym, h_sym, d_h, hd, d_hd, d2_hd],
                             **generator_kwargs)
    funcs_code[name] = code

    # generate the dynamic linearization intermediate control input
    mappings = {'q': m.q, 'eta': m.eta}
    printer = ArraySymbolsCodePrinter(mappings, **printer_kwargs)

    u = alg.Kdd_inv * (v_sym - alg.P)
    u_sym = matsym('u', m.eta)
    eta_sym = matsym('eta', m.eta)
    
    name = 'dynamic_lin_u'
    code = generate_function(name=prefix + name, 
                             printer=printer, 
                             out_expr=u, assign_to=u_sym, 
                             input_args=[v_sym, eta_sym, q_sym],
                             **generator_kwargs) 
    funcs_code[name] = code

#     # generate the dynamic linearization actual control
#     mappings = {'q': m.q, 'eta': m.eta}
#     printer = ArraySymbolsCodePrinter(mappings, **printer_kwargs)

#     u = alg.Kdd_inv * (v_sym - alg.P)
#     u_sym = matsym('u', m.eta)
#     eta_sym = matsym('eta', m.eta)

#     name = 'dynamic_lin_u'
#     code = generate_function(name=prefix + name, 
#                              printer=printer, 
#                              out_expr=u, assign_to=u_sym, 
#                              input_args=[v_sym, eta_sym, q_sym],
#                              unevaluated_args_dims=unevaluated_args_dims) 
#     funcs_code[name] = code


    # generate a function for computing the determinant of Kdd matrix
    det_Kdd = alg.Kdd.det().simplify()
    det_sym = Symbol('det_Kdd')

    mappings = {'q': m.q, 'eta': m.eta}
    printer = ArraySymbolsCodePrinter(mappings, **printer_kwargs)

    name = 'dynamic_lin_det_Kdd'
    code = """
float %s(const float q[], const float eta[]) {
    float %s
    return det_Kdd;
}
    """.strip() % (prefix + name, printer.doprint(det_Kdd, assign_to=det_sym))
    funcs_code[name] = code

    return funcs_code

def simplified_model_conversions_code(_model, _model_full, printer_kwargs, generator_kwargs, prefix=''):
    m, m_full = _model, _model_full

    funcs_code = {}
    
    # from full to simplified
    q = model.convert_simplified_to_full(m.q)  # the name is inverted in this situation
    q_simp_sym = matsym('q_simp', m.q)
    q_full_sym = matsym('q_full', m_full.q)

    mappings = {'q_simp': m.q, 'q_full': m_full.q}
    printer = ArraySymbolsCodePrinter(mappings, **printer_kwargs)

    name = 'full_to_simplified'
    code = generate_function(name=prefix + name, printer=printer, 
                             out_expr=q, assign_to=q_simp_sym, 
                             input_args=[q_full_sym],
                             **generator_kwargs) 
    funcs_code[name] = code
        
    # from simplified to full 
    q = model.convert_full_to_simplified(m_full.q)  # the name is inverted in this situation
    q_simp_sym = matsym('q_simp', m.q)
    q_full_sym = matsym('q_full', m_full.q)

    mappings = {'q_simp': m.q, 'q_full': m_full.q}
    printer = ArraySymbolsCodePrinter(mappings, **printer_kwargs)

    name = 'simplified_to_full'
    code = generate_function(name=prefix + name, printer=printer, 
                             out_expr=q, assign_to=q_full_sym, 
                             input_args=[q_simp_sym],
                             **generator_kwargs) 
    funcs_code[name] = code
    
    return funcs_code

# Code generation

In [5]:
# C99CodePrinter._default_settings =
# {'order': None,
#  'full_prec': 'auto',
#  'precision': 17,
#  'user_functions': {},
#  'human': True,
#  'allow_unknown_functions': False,
#  'contract': True,
#  'dereference': set(),
#  'error_on_reserved': False,
#  'reserved_word_suffix': '_'}

settings = lambda: {
    'type_aliases': {real: float32},
    # these need to be implemented (using trigonometric identities)
#     'user_functions': {'csc': 'csc', 'cot': 'cot'},
}

In [6]:
code_dir = './generated_code/'
if not os.path.exists(code_dir):
    os.makedirs(code_dir)
    
def print_funcs(funcs):
    for name, code in funcs.items():
        print(code)
        print()

In [7]:
# robot parameters
R = S(0.2)
l = S(0.5)
d = l/2

printer = ArraySymbolsCodePrinter({}, settings=settings())

with open(os.path.join(code_dir, 'robot_parameters.h'), 'w') as f:
    f.write('#pragma once' + 2*'\n')
    f.write('// Robot parameters definitions' + 2*'\n')   
    f.write('// radius of a sphere' + '\n')   
    f.write('#define R   %s'  % printer.doprint(R) + '\n')    
    f.write('// distance between the spheres' + '\n')   
    f.write('#define l   %s'  % printer.doprint(l) + '\n')
    f.write('// distance from the first sphere origin at which we want the robot' + '\n')   
    f.write('// to track the trajectory, required for simplified model' + '\n')   
    f.write('#define d   %s'  % printer.doprint(d) + '\n')

In [8]:
def func_call_wrapper(func_code):
    decl = function_declaration

## Static linearization

In [9]:
def gen_static_lin(prefix, model, algorithm, use_cse):
    funcs = static_lin_code(model, algorithm, 
                            printer_kwargs=dict(unevaluated_indexing=False, settings=settings()),
                            generator_kwargs=dict(unevaluated_args_dims=False, use_cse=use_cse), 
                            prefix=prefix)

    generate_file_pair(code_dir, prefix + 'static_lin', funcs.values(), 
                       includes=['#include <math.h>'], private_includes=['#include "robot_parameters.h"'])    
    return funcs

In [10]:
gen_static_lin('full_1_', models['full_1'], models['full_1_static'], use_cse=False)
gen_static_lin('full_1_cse_', models['full_1'], models['full_1_static'], use_cse=True);

gen_static_lin('full_2_', models['full_2'], models['full_2_static'], use_cse=False)
gen_static_lin('full_2_cse_', models['full_2'], models['full_2_static'], use_cse=True);

# impossible for full_2_JPTD because det(D) = 0

gen_static_lin('simplified_', models['simplified'], models['simplified_static'], use_cse=False)
gen_static_lin('simplified_cse_', models['simplified'], models['simplified_static'], use_cse=True);

## Dynamic linearization

In [11]:
def gen_dynamic_lin(prefix, model, algorithm, use_cse):
    funcs = dynamic_lin_code(model, algorithm, 
                             printer_kwargs=dict(unevaluated_indexing=False, settings=settings()),
                             generator_kwargs=dict(unevaluated_args_dims=False, use_cse=use_cse), 
                             prefix=prefix)

    generate_file_pair(code_dir, prefix + 'dynamic_lin', funcs.values(),    
                       includes=['#include <math.h>'], private_includes=['#include "robot_parameters.h"'])    
    
    return funcs

In [12]:
gen_dynamic_lin('full_1_', models['full_1'], models['full_1_dynamic'], use_cse=False)
gen_dynamic_lin('full_1_cse_', models['full_1'], models['full_1_dynamic'], use_cse=True);

gen_dynamic_lin('full_2_', models['full_2'], models['full_2_dynamic'], use_cse=False)
gen_dynamic_lin('full_2_cse_', models['full_2'], models['full_2_dynamic'], use_cse=True);

gen_dynamic_lin('full_2_JPTD_', models['full_2_JPTD'], models['full_2_JPTD_dynamic'], use_cse=False)
gen_dynamic_lin('full_2_JPTD_cse_', models['full_2_JPTD'], models['full_2_JPTD_dynamic'], use_cse=True);

gen_dynamic_lin('simplified_dyn1_', models['simplified'], models['simplified_dynamic_1'], use_cse=False)
gen_dynamic_lin('simplified_dyn1_cse_', models['simplified'], models['simplified_dynamic_1'], use_cse=True);

gen_dynamic_lin('simplified_dyn2_', models['simplified'], models['simplified_dynamic_2'], use_cse=False)
gen_dynamic_lin('simplified_dyn2_cse_', models['simplified'], models['simplified_dynamic_2'], use_cse=True);

## Simplified-full model conversions

In [13]:
def gen_simplified_model_conversions(prefix, model, model_full, use_cse):
    funcs = simplified_model_conversions_code(model, model_full, 
        printer_kwargs=dict(unevaluated_indexing=False, settings=settings()),
        generator_kwargs=dict(unevaluated_args_dims=False, use_cse=use_cse), 
        prefix=prefix)

    generate_file_pair(code_dir, prefix + 'conversions', funcs.values(),    
                       includes=['#include <math.h>'], private_includes=['#include "robot_parameters.h"'])    
    
    return funcs

# we can give any full model, only q is used
gen_simplified_model_conversions('simplified_', models['simplified'], models['full_2'], use_cse=False)
gen_simplified_model_conversions('simplified_cse_', models['simplified'], models['full_2'], use_cse=True);

[Warning] convert_simplified_to_full has not been verified
[Warning] convert_full_to_simplified has not been verified
[Warning] convert_simplified_to_full has not been verified
[Warning] convert_full_to_simplified has not been verified


## Examples

In [14]:
funcs = \
    gen_dynamic_lin('full_2_JPTD_cse_', models['full_2_JPTD'], models['full_2_JPTD_dynamic'], use_cse=True);

print_funcs(funcs)

void full_2_JPTD_cse_feedback_v(float v[5], const float K1[25], const float K2[25], const float q[9], const float h[5], const float d_h[5], const float hd[5], const float d_hd[5], const float d2_hd[5]) {
    float x0 = d_h[0] - d_hd[0];
    float x1 = d_h[1] - d_hd[1];
    float x2 = d_h[2] - d_hd[2];
    float x3 = d_h[3] - d_hd[3];
    float x4 = d_h[4] - d_hd[4];
    float x5 = hd[0] - h[0];
    float x6 = hd[1] - h[1];
    float x7 = hd[2] - h[2];
    float x8 = hd[3] - h[3];
    float x9 = hd[4] - h[4];
    
    v[0] = -K1[0]*x0 - K1[1]*x1 - K1[2]*x2 - K1[3]*x3 - K1[4]*x4 + K2[0]*x5 + K2[1]*x6 + K2[2]*x7 + K2[3]*x8 + K2[4]*x9 + d2_hd[0];
    v[1] = -K1[5]*x0 - K1[6]*x1 - K1[7]*x2 - K1[8]*x3 - K1[9]*x4 + K2[5]*x5 + K2[6]*x6 + K2[7]*x7 + K2[8]*x8 + K2[9]*x9 + d2_hd[1];
    v[2] = -K1[10]*x0 - K1[11]*x1 - K1[12]*x2 - K1[13]*x3 - K1[14]*x4 + K2[10]*x5 + K2[11]*x6 + K2[12]*x7 + K2[13]*x8 + K2[14]*x9 + d2_hd[2];
    v[3] = -K1[15]*x0 - K1[16]*x1 - K1[17]*x2 - K1[18]*x3 - K1[19]*x4 + K2[

In [15]:
funcs = simplified_model_conversions_code(models['simplified'], models['full_2'], 
                        printer_kwargs=dict(unevaluated_indexing=False, settings=settings()),
                        generator_kwargs=dict(unevaluated_args_dims=False, use_cse=False), 
                        prefix='')
for key, val in funcs.items():
    print('###', key)
    print(val)

[Warning] convert_simplified_to_full has not been verified
[Warning] convert_full_to_simplified has not been verified


### full_to_simplified
void full_to_simplified(float q_simp[9], const float q_full[9]) {
    
    q_simp[0] = q_simp[0];
    q_simp[1] = q_simp[1];
    q_simp[2] = q_simp[2];
    q_simp[3] = atan2f(sinf(q_full[4]), sinf(q_full[3])*cosf(q_full[4]));
    q_simp[4] = q_full[5];
    q_simp[5] = atan2f(sinf(q_full[7]), sinf(q_full[6])*cosf(q_full[7]));
    q_simp[6] = q_full[8];
    q_simp[7] = R*sqrtf((powf(sinf(q_full[4]), 2) - 1)*powf(cosf(q_full[3]), 2) + 1);
    q_simp[8] = R*sqrtf((powf(sinf(q_full[7]), 2) - 1)*powf(cosf(q_full[6]), 2) + 1);
}
### simplified_to_full
void simplified_to_full(float q_full[9], const float q_simp[9]) {
    
    q_full[0] = q_simp[0];
    q_full[1] = q_simp[1];
    q_full[2] = q_simp[2];
    q_full[3] = ((fmodf(q_simp[3], M_PI) > (3.0F/2.0F)*M_PI || fmodf(q_simp[3], M_PI) < M_PI_2) ? (
       1
    )
    : (
       -1
    ))*acosf(sqrtf(-powf(R, 2) + powf(q_simp[7], 2))*sqrtf(powf(tanf(q_simp[3]), 2) + 1)/sqrtf(-powf(R, 2)*powf(tanf(q_simp[3]), 2) - powf(R,